In [5]:
import os
import numpy as np
import pandas as pd
from scipy import interpolate
import glob

In [6]:
#identify mutiple events inside the same orbit

def Identify_Events(x):
    
    if (x.t.max() - x.t.min()) < 415:
    
        x['EVENT'] = 1
        return x
    
    else:
        
        t1 = x.t.iloc[0]
        x1 =  x[np.abs(x.t.values - t1) < 415]
        x1['EVENT'] = 1
        
        xc = x[np.abs(x.t.values - t1) >= 415]
        i = 1
        while(xc.size != 0):
            
            i += 1
            t2 = xc.t.iloc[0]
            x2 =  xc[np.abs(xc.t.values - t2) < 415]
            x2['EVENT'] = i
            x1 = pd.concat([x1,x2])
            xc = xc[np.abs(xc.t.values - t2) >= 415]
        
        return x1

In [7]:
#define the function that for esch orbit return the orbit with the values that have difference with the time of the 
#TDC_channel 137 or 138 less than the T_max 
#ridondance in the triggers (the sames in different ambiguity), will be solved later
#computing the drift time

#define the function that for esch orbit return the orbit with the values that have difference with the time of the 
#TDC_channel 139 less than the T_max 
#ridondance in the triggers (the sames in different ambiguity), will be solved later
#computing the drift time

def Time_Max_Filer(x):
    
        if (x.DETECTOR==0).sum() == 0:
            
            return None
            
        if (x.DETECTOR==0).sum() == 1:
        
            trigger_time = x[x.DETECTOR==0].t.values[0]
            x1 =  x[(x.t.values - trigger_time < 415)*(x.t.values - trigger_time > -25)]
            x1['AMBIGUITY'] = 1
            x1['DRIFT_TIME'] = x1.t.values - trigger_time
            
            return x1
            
        else:
                
            trigger_time = x[x.DETECTOR==0].t.values[0]
            x1 =  x[(x.t.values - trigger_time < 415)*(x.t.values - trigger_time > -25)]
            x1['AMBIGUITY'] = 1
            x1['DRIFT_TIME'] = x1.t.values - trigger_time
            
        
            i = 1
            for trigger_time in x[x.DETECTOR==0].t.values[1:]:
                i += 1
                x2 =  x[(x.t.values - trigger_time < 415)*(x.t.values - trigger_time > -25)]
                x2['AMBIGUITY'] = i
                x2['DRIFT_TIME'] = x2.t.values - trigger_time
                
                x1 = pd.concat([x1,x2])
                
            return x1

In [8]:
DataFrames = []

In [ ]:
for filename in glob.glob('../../data/Run000260/*')[:1]:
    
    try:
        data = pd.read_csv(filename)

        #adding time column
        data['t'] = data['BX_COUNTER']*25 + data['TDC_MEAS']*25/30


        #List of conditions to assign a hit to a detector
        fpga = data['FPGA'] == 1 
        channel = (data['TDC_CHANNEL'] > 64) & (data['TDC_CHANNEL'] <= 128) #condition on TDC_CHANNEL
        trigger = data['TDC_CHANNEL'] >= 129 

        conditions = [trigger, ~fpga & ~channel, ~fpga & channel, fpga & ~channel, fpga & channel]
        values = [0, 1, 2, 3, 4]

        #adding detector column
        data['DETECTOR'] = np.select(conditions, values)


        #List of conditions to assign a hit to a layer
        layer_4 = data['TDC_CHANNEL']%4 == 0 
        layer_3 = (data['TDC_CHANNEL'] + 2) % 4 == 0
        layer_2 = (data['TDC_CHANNEL'] + 1) % 4 == 0
        layer_1 = (data['TDC_CHANNEL'] + 3) % 4 == 0
        trigger = data['TDC_CHANNEL'] >= 129 

        #adding layer column
        conditions = [trigger, layer_4, layer_3, layer_2, layer_1]
        values = [0, 4, 3, 2, 1]

        data['LAYER'] = np.select(conditions, values)

        #add column coordinate of the cell
        data['COLUMN'] = ((data['TDC_CHANNEL'] % 64 
                   - interpolate.BarycentricInterpolator([1,2,3,4],[1,3,2,4])(data['LAYER'])) / 4 + 1).astype(int)

        #set column value to 0 when trigger is activate
        data.loc[data['DETECTOR'] == 0, 'COLUMN'] = 0
    
        #i consider orbit only with scintillators, i drop other trigger raws
        data_139 = data.groupby('ORBIT_CNT', group_keys=False).filter(lambda x: 139 in x.TDC_CHANNEL.values)
        data_139 = data_139[data_139['TDC_CHANNEL'] != 130]
        data_139 = data_139[data_139['TDC_CHANNEL'] != 129]
        data_139 = data_139[data_139['TDC_CHANNEL'] != 138]
        data_139 = data_139[data_139['TDC_CHANNEL'] != 137]

        #adding events column
        data_139 = data_139.groupby('ORBIT_CNT', group_keys=False).apply(Identify_Events)
        
        #filtering again events for trigger 139 activation
        data_139 = data_139.groupby(['ORBIT_CNT','EVENT'], group_keys=False).filter(lambda x: 139 in x.TDC_CHANNEL.values)

        #look at the function
        data_139 = data_139.groupby(['ORBIT_CNT', 'EVENT'], group_keys=False).apply(Time_Max_Filer)

        #dropping trigger events
        data_139 = data_139[data_139['DETECTOR'] != 0]

        #dropping events with number of hits hit less than 3 (ambiguity not solvable)
        data_139 = data_139.groupby(['ORBIT_CNT', 'EVENT', 'AMBIGUITY', 'DETECTOR']).filter(lambda x: len(x) >= 3)

        #dropping some columns
        data_139 = data_139.drop(['HEAD', 'FPGA', 'TDC_CHANNEL', 'BX_COUNTER', 'TDC_MEAS'], axis = 1)
        
        DataFrames.append(data_139)
        
        
    except:
        None

In [ ]:
pd.concat(DataFrames)

In [7]:
pd.concat(DataFrames).to_csv('../../data/Run000260/data_139.txt', index = False)

In [10]:
pd.read_csv('../../data/Run000260/data_139.txt')

,ORBIT_CNT,t,DETECTOR,LAYER,COLUMN,EVENT,AMBIGUITY,DRIFT_TIME
0,1933476862,52487.500000,3,1,6,1,1,112.500000
1,1933476862,52525.000000,3,4,8,1,1,150.000000
2,1933476862,52429.166667,3,1,7,1,1,54.166667
3,1933476946,65314.166667,3,2,14,1,1,39.166667
4,1933476946,65410.000000,3,1,15,1,1,135.000000
5,1933476946,65443.333333,3,3,15,1,1,168.333333
6,1933476976,84795.833333,3,1,2,2,1,45.833333
7,1933476976,84821.666667,3,2,2,2,1,71.666667
8,1933476976,84841.666667,4,2,4,2,1,91.666667
9,1933476976,84860.833333,3,4,2,2,1,110.833333
